In [16]:
import csv
import os
from datetime import datetime
import json
import psycopg2
import sys
from ipyleaflet import Map, WKTLayer
import re

sys.path.insert(0,'..')

from scripts.database import get_conn_pool, execute, fetch_all
from scripts.routing import route_nogo, route_standard

In [17]:
# connect to postgis database
pool = get_conn_pool({
    'host': 'localhost',
    'port': 5432,
    'user': 'postgres',
    'password': 'postgres',
    'database': 'xlab-routing'
})

In [18]:
# create/define pgr_nogo_dijkstra_random function in database
nogo_function_path = "../scripts/define_nogo_dijkstra_random.sql"
nogo_fd = open(nogo_function_path, 'r')
execute(pool, nogo_fd.read())

# create/define pgr_dijkstra_random function in database
function_path = "../scripts/define_dijkstra_random.sql"
fd = open(function_path, 'r')
execute(pool, fd.read())

In [19]:
# set start and end points

# START COORDINATES (enter long, lat)
start = [36.78254,-1.33091]
start_lat = start[1]
start_long = start[0]

# END COORDINATES (enter long, lat)
end = [36.73858,-1.16696]
end_lat = end[1]
end_long = end[0]

In [20]:
# set parameters

# nogo layers
nogo_layer_list = ['hgis_admin3', 'hgis_bridges_tunnels']

# nogo layer queries (needs to be in same order as nogo_layer_list)
nogo_layer_query_list = ["hgis_admin3.adm3_name = 'Kitisuru'", 
                         "hgis_bridges_tunnels.objectid = 810 OR hgis_bridges_tunnels.objectid = 811"]

# nogo point layer buffer distance (default is .0005 in degrees or 50 meters) 
nogo_point_buffer = .0005

# nogo polyline layer buffer distance (default is .0001 in degrees or 10 meters)
nogo_line_buffer = .0001

# nogo cost (default is 1000000000)
nogo_cost = 1000000000

# noise for randomness (default is 0 (no noise) which gives original cost and doesn't randomize)
noise = .75


In [21]:
# get nogo route + cost randomness
try:
    print("NOGO ROUTE")
    print("")
    route_nogo = route_nogo(nogo_layer_list, start_long, start_lat, end_long, end_lat, nogo_layer_query_list, nogo_point_buffer, nogo_line_buffer, nogo_cost, noise)
    nogo_wkt = route_nogo[0]
    nogo_length = route_nogo[1]
except:
    pass

# get standard route + cost randomness
try:
    print("")
    print("STANDARD ROUTE")
    print("")
    route_standard = route_standard(start_long, start_lat, end_long, end_lat, noise)
    wkt = route_standard[0]
    length = route_standard[1]
except:
    pass

NOGO ROUTE

NOGO QUERY:  SELECT ST_GEOMFROMEWKT(ST_COLLECT(a.shape)) FROM (SELECT ST_FORCE2D(shape) as shape FROM hgis_admin3 WHERE hgis_admin3.adm3_name = 'Kitisuru' UNION SELECT ST_FORCE2D(ST_BUFFER(shape, 0.0005)) as shape FROM hgis_bridges_tunnels WHERE hgis_bridges_tunnels.objectid = 810 OR hgis_bridges_tunnels.objectid = 811) as a
START NODE:  8543
END NODE:  7889
LENGTH:  30463.476544109373

STANDARD ROUTE

START NODE:  8543
END NODE:  7889
LENGTH:  27197.101958558032


In [22]:
# Display route

# center map
m = Map(center=[start_lat, end_long], zoom=10)

# nogo layer 
nogo_wlayer = WKTLayer(
    wkt_string=nogo_wkt,
    hover_style={"color": "red"},
    style={"color": "blue"}
)
m.add_layer(nogo_wlayer)

# standard layer
wlayer = WKTLayer(
    wkt_string=wkt,
    hover_style={"color": "pink"},
    style={"color": "green"}
)
m.add_layer(wlayer)

m

Map(center=[-1.33091, 36.73858], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…